## install & load packages

In [ ]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyddm import Sample

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# 8 drifts, different effects for congruent & incongruent

In [ ]:
def drift(t, trueCongruence, signal1_onset, noise2_onset, signal2_onset,
                noise1_cong, noise1_incong, noise1_neut, signal1_cong, signal1_incong, signal1_neut,
                noise2_cong, noise2_incong, noise2_neut, signal2_cong, signal2_incong, signal2_neut):
  # drift rate during first noise period
  if t < signal1_onset:
    if trueCongruence == 'congruent':
      return noise1_cong
    elif trueCongruence == 'incongruent':
      return -noise1_incong
    else:
      return noise1_neut

  # drift rates during first signal period
  if t >= signal1_onset and t < noise2_onset:
    if trueCongruence == 'congruent':
      return signal1_cong
    elif trueCongruence == 'incongruent':
      return -signal1_incong
    else:
      return signal1_neut

  # drift rates during the second noise period
  if t >= noise2_onset and t < signal2_onset:
    if trueCongruence == 'congruent':
      return noise2_cong
    elif trueCongruence == 'incongruent':
      return -noise2_incong
    else:
      return noise2_neut

  # drift rates during the second signal period
  if t >= signal2_onset:
    if trueCongruence == 'congruent':
      return signal2_cong
    elif trueCongruence == 'incongruent':
      return -signal2_incong
    else:
      return signal2_neut

In [72]:
model = pyddm.gddm(
    drift = drift,
    starting_position = 0,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'B': (0.5, 10), 'ndt': (0.01, 1.5),
                'noise1_cong': (0, 20), 'noise1_incong': (0,20), 'noise1_neut': (0,20),
                'signal1_cong': (0, 20), 'signal1_incong': (0, 20), 'signal1_neut': (0, 20),
                'noise2_cong': (0, 20), 'noise2_incong': (0,20), 'noise2_neut': (0,20),
                'signal2_cong': (0, 20), 'signal2_incong': (0, 20), 'signal2_neut': (0,20)},
    conditions = ['trueCongruence', 'signal1_onset', 'noise2_onset', 'signal2_onset']
)

In [69]:
# Load data - need to manually upload inference_tidy.csv into the files tab on each new runtime
df = pd.read_csv('inference.csv')
df = df.dropna(subset=['RT'])
df[['signal1_onset', 'noise2_onset', 'signal2_onset']] = df[['signal1_onset', 'noise2_onset', 'signal2_onset']].fillna(0)
#df['signal1_duration'] = df['noise2_onset'] - df['signal1_onset']


sub_df = df[df['subID'] == 33]
sample = pyddm.Sample.from_pandas_dataframe(sub_df, rt_column_name='RT', choice_column_name='accuracy')

In [73]:
pyddm.plot.model_gui_jupyter(model, sample, data_dt = 0.04)

Output()

# try binning durations

In [ ]:
df = pd.read_csv('inference.csv')
df = df.dropna(subset=['RT'])

sub_df = df[df['subID'] == 33]
sub_df.replace(np.nan, 0, inplace=True)
sample = pyddm.Sample.from_pandas_dataframe(sub_df, rt_column_name='RT', choice_column_name='accuracy')